In [113]:
import pandas as pd 
import numpy as np  
import ast
import pyarrow
from textblob import TextBlob

from sklearn.metrics.pairwise        import cosine_similarity
from sklearn.metrics.pairwise        import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

## LECTURA DE LOS ARCHIVOS

In [137]:
df = pd.read_json('steam_games.json', orient='records', lines=True, convert_dates=True)
df.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


In [117]:
lista = []
df2 = '/Users/danielh/Desktop/P HENRY/GITHUB/PROY-1-IND-HENRY/users_items.json'


#Abrir el archivo y procesar cada línea
with open(df2, 'r', encoding='utf-8') as df:
    for i in df:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(i)
            lista.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {i}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
df2 = pd.DataFrame(lista)

In [118]:
df2.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [119]:
lista = []
df3 = 'user_reviews.json'

#Abrir el archivo y procesar cada línea
with open(df3, 'r', encoding='utf-8') as df:
    for i in df:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(i)
            lista.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {i}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
df3 = pd.DataFrame(lista)

In [54]:
df3.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


# PREPARANDO EL PRIMER ARCHIVO

In [138]:
df.rename(columns = {'id':'item_id'}, inplace = True)


##### Borrando columnas inecesarias

In [139]:
df1 = df.drop(columns=['app_name', 'url', 'reviews_url', 'specs', 'early_access'], axis=1)

In [143]:
df1["genres"]=df1["genres"].astype(str)

##### Limpiando columna genres ya que sera utilizada en los enpoint

In [145]:
df1["genres"]=df1['genres'].str.strip("[]").str.replace("'", "")

#### Borrando datos faltantes, para ahorrar espacio se  aplico a todo el Dataframe y no se ocupo axis=1

In [149]:
df1 = df1.dropna()

Reemplazando los datos que son FREE como free to Play por 0  para  su utilizacion mas adelante 

In [150]:
def convertir_a_entero_o_cero(df1):
    try:
        return float(df1)
    except (ValueError, TypeError):
        return 0

# Aplicar la función a la columna 'columna_strings' del DataFrame
df1['price'] = df1['price'].apply(convertir_a_entero_o_cero)

In [151]:
df1.head(3)

,publisher,genres,title,release_date,tags,price,item_id,developer
88310,Kotoshiro,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
88311,"Making Fun, Inc.","Free to Play, Indie, RPG, Strategy",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",0.00,643980.0,Secret Level SRL
88312,Poolians.com,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",0.00,670290.0,Poolians.com


Convirtiendo la columna release_date en formato fecha para su utilizacion futura

In [152]:
df1['release_date'] = pd.to_datetime(df1['release_date'], errors='coerce')

## Preparacion del 2 Dataframe

In [121]:
df2.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')

In [122]:
df2.shape

(88310, 5)

Desanidando la columna items del Dataframe

In [123]:
df2= df2.explode(['items'])
ddf2 = pd.json_normalize(df2['items']).set_index(df2['items'].index)
df02= pd.concat([df2, ddf2], axis=1)

In [125]:
df02 = df02.drop(columns=['item_id', 'items', 'user_url'])

In [126]:
df02.tail(3)

,user_id,items_count,steam_id,item_name,playtime_forever,playtime_2weeks
88308,76561198329548331,7,76561198329548331,You Have 10 Seconds 2,4.0,4.0
88308,76561198329548331,7,76561198329548331,Minds Eyes,3.0,3.0
88309,edward_tremethick,0,76561198331598578,NaN,NaN,NaN


In [34]:
df02["item_id"]= df02["item_id"].astype(float)

In [35]:
df02.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5170015 entries, 0 to 88309
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   items             object 
 5   item_id           float64
 6   item_name         object 
 7   playtime_forever  float64
 8   playtime_2weeks   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 394.4+ MB


# Modificacion de 3 dataframe

In [127]:
df3= df3.explode(['reviews'])
ddf3 = pd.json_normalize(df3['reviews']).set_index(df3['reviews'].index)
df03= pd.concat([df3, ddf3], axis=1)

In [129]:
df03 = df03.drop(columns=['user_url', 'funny', 'posted', 'helpful', 'last_edited', 'reviews'])

In [130]:
df03.tail(3)

,user_id,item_id,recommend,review
25798,LydiaMorley,273110,True,had so much fun plaing this and collecting res...
25798,LydiaMorley,730,True,:D
25798,LydiaMorley,440,True,so much fun :D


### ANALISIS DE SENTIMIENTOS

In [131]:
def analizar_sentimiento(text):
    sentimiento = TextBlob(text)
    return sentimiento.sentiment.polarity
       

CREACION DE COLUMNA DE ANALISIS DE SENTIMIENTO

In [132]:
df03['review'] = df03['review'].astype(str)

df03['Polaridad del sentimiento'] = df03['review'].apply(analizar_sentimiento)

df03['Sentimiento'] = df03['Polaridad del sentimiento'].apply(
    lambda x: 2 if x > 0 else (1 if x == 0 else 0)
)


In [133]:
df03.tail(2)

,user_id,item_id,recommend,review,Polaridad del sentimiento,Sentimiento
25798,LydiaMorley,730,True,:D,1.00,2
25798,LydiaMorley,440,True,so much fun :D,0.65,2


### UNION DEL SEGUNDO Y TERCER DATAFRAME 

In [134]:
df123= pd.merge(df02, df03, on='user_id')

In [135]:
df123.head(2)

,user_id,items_count,steam_id,item_name,playtime_forever,playtime_2weeks,item_id,recommend,review,Polaridad del sentimiento,Sentimiento
0,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,1250,True,Simple yet with great replayability. In my opi...,0.174444,2
1,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,22200,True,It's unique and worth a playthrough.,0.337500,2


In [157]:
df123['item_id']=df123['item_id'].astype(float)

# UNIR EL DATAFRAME UNIDO ANTERIORMENTE, CON EL DATAFRAME 1

In [158]:
DataSet_Final = pd.merge(df123, df1, on='item_id')

CREACION DE UNA COLUMNA A FORMATO FECHA PARA UN USO POSTERIO DE LA COLUMNA CREADA

In [159]:
DataSet_Final['anio'] = pd.to_datetime(DataSet_Final["release_date"], errors='coerce')

In [161]:
DataSet_Final.reset_index(drop=True)

,user_id,items_count,steam_id,item_name,playtime_forever,playtime_2weeks,item_id,recommend,review,Polaridad del sentimiento,Sentimiento,publisher,genres,title,release_date,tags,price,developer,anio
0,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,1250.0,True,Simple yet with great replayability. In my opi...,0.174444,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
1,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,22200.0,True,It's unique and worth a playthrough.,0.337500,2,ACE Team,"Action, Indie",Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009-04-21
2,76561197970982479,277,76561197970982479,Team Fortress Classic,0.0,0.0,1250.0,True,Simple yet with great replayability. In my opi...,0.174444,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
3,76561197970982479,277,76561197970982479,Team Fortress Classic,0.0,0.0,22200.0,True,It's unique and worth a playthrough.,0.337500,2,ACE Team,"Action, Indie",Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009-04-21
4,76561197970982479,277,76561197970982479,Day of Defeat,7.0,0.0,1250.0,True,Simple yet with great replayability. In my opi...,0.174444,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6454903,76561198312638244,36,76561198312638244,DOOM,1178.0,0.0,70.0,True,a must have classic from steam definitely wort...,0.233333,2,Valve,Action,Half-Life,1998-11-08,"[FPS, Classic, Action, Sci-fi, Singleplayer, S...",9.99,Valve,1998-11-08
6454904,76561198312638244,36,76561198312638244,DOOM,1178.0,0.0,362890.0,True,this game is a perfect remake of the original ...,0.142500,2,Crowbar Collective,"Action, Adventure, Indie, Early Access",Black Mesa,2015-05-05,"[Early Access, FPS, Remake, Action, Sci-fi, Fi...",19.99,Crowbar Collective,2015-05-05
6454905,LydiaMorley,0,76561198313816521,NaN,NaN,NaN,273110.0,True,had so much fun plaing this and collecting res...,0.087500,2,Nexon Europe GmbH,"Action, Casual, Free to Play, Massively Multip...",Counter-Strike Nexon: Zombies,2014-10-07,"[Free to Play, Zombies, Multiplayer, FPS, Shoo...",0.00,Nexon,2014-10-07
6454906,LydiaMorley,0,76561198313816521,NaN,NaN,NaN,730.0,True,:D,1.000000,2,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,"[FPS, Multiplayer, Shooter, Action, Team-Based...",14.99,Valve,2012-08-21


##### Borrar columnas que no se van a utilizar y no han sido borradas

In [162]:
DataSet_Final = DataSet_Final.drop(columns=['tags', 'release_date'])

### Creacion de columna year para los endpoints, y medicion del tiempo

In [163]:
DataSet_Final['anio'] = DataSet_Final['anio'].dt.year

##### Comprobar si ya esta listo el Dataset

In [164]:
DataSet_Final.tail(15)

,user_id,items_count,steam_id,item_name,playtime_forever,playtime_2weeks,item_id,recommend,review,Polaridad del sentimiento,Sentimiento,publisher,genres,title,price,developer,anio
6454893,76561198312638244,36,76561198312638244,ShareX,1.0,0.0,233270.0,True,this is a very fun and nice 80s themed shooter...,0.249091,2,Ubisoft,"Action, Adventure",Far Cry 3 - Blood Dragon,14.99,Ubisoft Montreal,2013.0
6454894,76561198312638244,36,76561198312638244,ShareX,1.0,0.0,130.0,True,if you liked Half life i would really recommen...,0.193333,2,Valve,Action,Half-Life: Blue Shift,4.99,Gearbox Software,2001.0
6454895,76561198312638244,36,76561198312638244,ShareX,1.0,0.0,70.0,True,a must have classic from steam definitely wort...,0.233333,2,Valve,Action,Half-Life,9.99,Valve,1998.0
6454896,76561198312638244,36,76561198312638244,ShareX,1.0,0.0,362890.0,True,this game is a perfect remake of the original ...,0.142500,2,Crowbar Collective,"Action, Adventure, Indie, Early Access",Black Mesa,19.99,Crowbar Collective,2015.0
6454897,76561198312638244,36,76561198312638244,Devil Daggers,0.0,0.0,233270.0,True,this is a very fun and nice 80s themed shooter...,0.249091,2,Ubisoft,"Action, Adventure",Far Cry 3 - Blood Dragon,14.99,Ubisoft Montreal,2013.0
6454898,76561198312638244,36,76561198312638244,Devil Daggers,0.0,0.0,130.0,True,if you liked Half life i would really recommen...,0.193333,2,Valve,Action,Half-Life: Blue Shift,4.99,Gearbox Software,2001.0
6454899,76561198312638244,36,76561198312638244,Devil Daggers,0.0,0.0,70.0,True,a must have classic from steam definitely wort...,0.233333,2,Valve,Action,Half-Life,9.99,Valve,1998.0
6454900,76561198312638244,36,76561198312638244,Devil Daggers,0.0,0.0,362890.0,True,this game is a perfect remake of the original ...,0.142500,2,Crowbar Collective,"Action, Adventure, Indie, Early Access",Black Mesa,19.99,Crowbar Collective,2015.0
6454901,76561198312638244,36,76561198312638244,DOOM,1178.0,0.0,233270.0,True,this is a very fun and nice 80s themed shooter...,0.249091,2,Ubisoft,"Action, Adventure",Far Cry 3 - Blood Dragon,14.99,Ubisoft Montreal,2013.0
6454902,76561198312638244,36,76561198312638244,DOOM,1178.0,0.0,130.0,True,if you liked Half life i would really recommen...,0.193333,2,Valve,Action,Half-Life: Blue Shift,4.99,Gearbox Software,2001.0


# Guardar Dataset a parquet para ahorrar espacio

In [26]:
DataSet_Final.to_parquet("DataSet_Final2.parquet")

#### Lectura y creacion archivos individuales para los endpoint para ahorrar espacio y acelerar las consultas

In [ ]:
df=pd.read_parquet('DataSet_Final2.parquet', engine="pyarrow")


df[['developer', "anio", "items_count", "price"]].to_parquet("df.parquet")
df[['user_id', "item_id", "recommend", "price"]].to_parquet("df2.parquet")
df[['genres', "user_id", "playtime_forever", "anio"]].to_parquet("df3.parquet")
df[['anio', "developer"]].to_parquet("df4.parquet")
df[['developer', "Sentimiento"]].to_parquet("df5.parquet")